200学生100题情况下，最佳w值使误差最小。使用pymc所求难度b。

In [1]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')
np.random.seed(1234)
import pymc
import scipy.stats as stats
from scipy.optimize import minimize, rosen, rosen_der

In [14]:
n_items=200
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
mask=np.random.random(uv.shape)<0.7
def absm(a,b):
    return np.abs(a-b).mean()

def uirt(uv,mask):
    n_stu,n_items=uv.shape    
    # 难度参数
    b1=pymc.Normal('b1',mu=0,tau=1,value=np.zeros((1,n_items,)))
    # 能力参数
    theta1=pymc.Normal('theta1',mu=0,tau=1,value=np.zeros((n_stu,1)))
    # 难度参数
    b2=pymc.Normal('b2',mu=0,tau=1,value=np.zeros((1,n_items,)))
    # 能力参数
    theta2=pymc.Normal('theta2',mu=0,tau=1,value=np.zeros((n_stu,1)))
    @pymc.deterministic
    def sigmoid(theta1=theta1,b1=b1,theta2=theta2,b2=b2): 
        return 1.0 / (1.0 + np.exp(-(theta1-b1)))/ (1.0 + np.exp(-(theta2-b2)))
    # 答题矩阵
    u=pymc.Bernoulli('u', p=sigmoid, value=uv, observed=True) 
    params=[b1,theta1,b2,theta2,sigmoid]
    params.append(u)      
    m=pymc.MCMC(params)
    # 似然函数
#     print u.logp
    m.sample(200000,40000,12)
#     print u.logp
    eb1=np.array([b1.stats()['mean']])
    et1=np.array([theta1.stats()['mean']]).T
    eb2=np.array([b2.stats()['mean']])
    et2=np.array([theta2.stats()['mean']]).T
    m.stats()
    # 误差
    print absm(eb1.reshape(real_b1.shape),real_b1),absm(et1.reshape(real_theta1.shape),real_theta1)
    print absm(eb2.reshape(real_b2.shape),real_b2),absm(et2.reshape(real_theta2.shape),real_theta2)
    return eb1,et1,eb2,et2
eb1,et1,eb2,et2=uirt(uv,mask)
k = 0.5


 [-----------------100%-----------------] 200000 of 200000 complete in 1045.4 secCould not generate output statistics for sigmoid
0.844327813535 0.812759933328
0.866010818214 0.877301986113


In [18]:
uv_test=np.ones((n_stu,n_items))*k<1.0/ (1.0 + np.exp(-(et1-eb1)))/ (1.0 + np.exp(-(et2-eb2)))
result=uv_test==uv
# 正确率
print result.sum()/n_items/n_stu

0.77245


In [17]:
b0 = np.array([ee for ee in eb1.flat])
b1 = np.array([ee for ee in eb2.flat])
bnds = ((-3, 3), (-3, 3))
for w in [ii/100. for ii in range(100)]:
    r_fun = 0
    su = 0
    ab1 = []
    ab2 = []
    theta_min1 = []
    theta_min2 = []
    for i in range(n_stu):
        y = np.array([ee for ee in uv[i].flat])
        fun = lambda x: sum(np.log(1+ np.exp(-(x[0]-b0)))+np.log(1+ np.exp(-(x[1]-b1)))-(1-y)*np.log((1+ np.exp(-(x[0]-b0)))*(1+ np.exp(-(x[1]-b1)))-1))/n_items+w*(x[0]**2+x[1]**2)
        res = minimize(fun, (et1[i][0],et2[i][0]), bounds=bnds,method='L-BFGS-B')
        r_fun += res.fun 
        # 存储t
        theta_min1.append(res.x[0])
        theta_min2.append(res.x[1])
        # 存储t的差值
        ab1.append(real_theta1[i][0]-res.x[0])
        ab2.append(real_theta2[i][0]-res.x[1])
        result = k<1.0/ (1.0 + np.exp(-(res.x[0]-b0)))/ (1.0 + np.exp(-(res.x[1]-b1)))
        su += sum(result==y)
    print w,su*1./n_items/n_stu,sum([abs(aa) for aa in ab1])*1./n_stu+sum([abs(aa) for aa in ab2])/n_stu,sum([abs(aa) for aa in ab1])*1./n_stu,sum([abs(aa) for aa in ab2])/n_stu,r_fun

0.0 0.77415 3.01643442241 1.42214018811 1.5942942343 46.6754088583
0.01 0.77235 1.51978285031 0.727430280348 0.792352569963 48.2708914888
0.02 0.7687 1.37019785265 0.647296845307 0.722901007346 49.0150458943
0.03 0.76705 1.32904510888 0.62435363751 0.704691471373 49.5511615694
0.04 0.76435 1.32019662993 0.620455229918 0.699741400014 49.9706002489
0.05 0.7628 1.32774706895 0.621902643386 0.705844425568 50.3120876682
0.06 0.762 1.33952301459 0.625925150367 0.71359786422 50.5972358348
0.07 0.76085 1.35159004251 0.630347608188 0.721242434327 50.8397358092
0.08 0.75995 1.36296030461 0.634691495755 0.728268808851 51.0489156081
0.09 0.7589 1.37429338557 0.639433632739 0.734859752833 51.2314431486
0.1 0.757 1.38518620623 0.644240921523 0.740945284702 51.3922530868
0.11 0.75555 1.39514424982 0.648665058867 0.746479190949 51.535096023
0.12 0.75435 1.40434960265 0.65282630267 0.751523299975 51.6628842667
0.13 0.75355 1.41312253856 0.65699124604 0.756131292519 51.7779195975
0.14 0.75195 1.42119470